<a href="https://colab.research.google.com/github/annvorosh/GB/blob/NLP/NLP_L05_Pos_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тема «POS-tagger и NER»

## Задание 1.
- Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы, сделать выводы  


## Задание 2.
- Проверить, насколько хорошо работает NER
- Данные брать из Index of /pub/named_entities
- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
- передаём в сетку токен и его соседей.
- передаём в сетку только токен.
- свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

In [7]:
!pip install corus

In [8]:
!wget https://github.com/UniversalDependencies/UD_Russian-PUD/raw/master/ru_pud-ud-test.conllu

--2023-07-28 12:29:00--  https://github.com/UniversalDependencies/UD_Russian-PUD/raw/master/ru_pud-ud-test.conllu
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-PUD/master/ru_pud-ud-test.conllu [following]
--2023-07-28 12:29:00--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-PUD/master/ru_pud-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1867908 (1.8M) [text/plain]
Saving to: ‘ru_pud-ud-test.conllu.2’

ru_pud-ud-test.conl 100%[===================>]   1.78M  --.-KB/s    in 0.07s   

2023-07-28 12:29:00 (24.1 MB/s) - ‘ru_pud-

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
from corus import load_ud_pud

path = 'ru_pud-ud-test.conllu'
records = load_ud_pud(path)
# next(records)

In [15]:
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger, PerceptronTagger
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


In [16]:
# Получите предложения с разметкой частей речи
sentences = [[(token.text, token.pos) for token in record.tokens] for record in records]

# Разделите данные на обучающую и тестовую выборки
train_size = int(0.8 * len(sentences))
train_sent = sentences[:train_size]
test_sent = sentences[train_size:]

In [18]:
# Создание и обучение теггеров
unigram_tagger = UnigramTagger(train_sent)
bigram_tagger = BigramTagger(train_sent, backoff=unigram_tagger)
trigram_tagger = TrigramTagger(train_sent, backoff=bigram_tagger)

# Оценка производительности теггеров на тестовых данных
print("UnigramTagger accuracy:", unigram_tagger.evaluate(test_sent))
print("BigramTagger accuracy:", bigram_tagger.evaluate(test_sent))
print("TrigramTagger accuracy:", trigram_tagger.evaluate(test_sent))

UnigramTagger accuracy: 0.6213304605440345
BigramTagger accuracy: 0.6218691085375707
TrigramTagger accuracy: 0.6226770805278751


In [52]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN')
tag = backoff_tagger(train_sent,
                     [UnigramTagger, TrigramTagger, BigramTagger],
                     backoff = backoff)

tag.evaluate(test_sent)

0.7613789388634528

Комбинация различных теггеров позволяет использовать их преимущества и улучшает результаты. Если один из теггеров не может определить тег для слова, резервный теггер DefaultTagger использует тег 'NOUN', что является общим предположением.
Порядок, в котором теггеры используются как резервные, может повлиять на результаты, и изменение этого порядка может привести к небольшому увеличению точности. В нашем случае порядок [UnigramTagger, TrigramTagger, BigramTagger] показал наилучший результат.

In [26]:
perceptron_tagger = PerceptronTagger()
perceptron_tagger.train(train_sent)
print("PerceptronTagger accuracy:", perceptron_tagger.evaluate(test_sent))

PerceptronTagger accuracy: 0.9081605171020738


### ВЫВОД:
PerceptronTagger показывает гораздо более высокую точность в сравнении с другими теггерами, так как он использует статистические методы для обучения на размеченных данных.

- PUNCT: знаки пунктуации
- AUX: вспомогательные глаголы
- NOUN: существительные
- VERB: глаголы
- SCONJ: подчинительные союзы
- PRON: местоимения
- ADJ: прилагательные

In [53]:
patterns = [
    # Глаголы
    (r'.*ть$', 'VERB'),            # инфинитивы на -ть
    (r'.*ать$', 'VERB'),           # инфинитивы на -ать
    (r'.*ить$', 'VERB'),           # инфинитивы на -ить
    (r'.*еть$', 'VERB'),           # инфинитивы на -еть
    (r'.*аться$', 'VERB'),        # возвратные глаголы на -ся
    (r'.*сь$', 'VERB'),           # возвратные глаголы на -сь
    (r'.*уся$', 'VERB'),          # возвратные глаголы на -уся
    (r'.*и$', 'VERB'),             # глаголы на -и
    (r'.*ется$', 'VERB'),          # 3-е лицо ед.ч. наст.вр. глаголов на -ить и -еть
    (r'.*ем$', 'VERB'),            # 1-е лицо мн.ч. наст.вр. глаголов на -ить и -еть
    (r'.*ает$', 'VERB'),           # 3-е лицо ед.ч. наст.вр. глаголов на -ать
    (r'.*ают$', 'VERB'),           # 3-е лицо мн.ч. наст.вр. глаголов на -ать
    (r'.*яется$', 'VERB'),         # 3-е лицо ед.ч. наст.вр. возвратных глаголов на -иться и -еться
    (r'.*емся$', 'VERB'),          # 1-е лицо мн.ч. наст.вр. возвратных глаголов на -иться и -еться
    (r'.*аются$', 'VERB'),         # 3-е лицо мн.ч. наст.вр. возвратных глаголов на -аться
    (r'.*ится$', 'VERB'),          # 3-е лицо ед.ч. наст.вр. глаголов на -ить
    (r'.*ет$', 'VERB'),            # 3-е лицо ед.ч. наст.вр. глаголов на -еть

    # Существительные
    (r'.*ия$', 'NOUN'),            # существительные на -ия
    (r'.*ость$', 'NOUN'),          # существительные на -ость
    (r'.*ие$', 'NOUN'),            # существительные на -ие
    (r'.*ки$', 'NOUN'),            # существительные на -ки
    (r'.*ы$', 'NOUN'),             # существительные на -ы
    (r'.*а$', 'NOUN'),             # существительные на -а
    (r'.*я$', 'NOUN'),             # существительные на -я
    (r'.*о$', 'NOUN'),             # существительные на -о
    (r'.*у$', 'NOUN'),             # существительные на -у
    (r'.*е$', 'NOUN'),             # существительные на -е
    (r'.*сть$', 'NOUN'),           # существительные на -сть
    (r'.*ечка$', 'NOUN'),          # существительные на -ечка
    (r'.*ище$', 'NOUN'),           # существительные на -ище

    # Прилагательные
    (r'.*ый$', 'ADJ'),           # прилагательные на -ый
    (r'.*ий$', 'ADJ'),           # прилагательные на -ий
    (r'.*ая$', 'ADJ'),           # прилагательные на -ая
    (r'.*ое$', 'ADJ'),           # прилагательные на -ое
    (r'.*ые$', 'ADJ'),           # прилагательные на -ые
    (r'.*ь$', 'ADJ'),            # прилагательные на -ь
    (r'.*ого$', 'ADJ'),          # прилагательные на -ого
    (r'.*его$', 'ADJ'),          # прилагательные на -его

    # Наречия
    (r'.*о$', 'ADV'),            # наречия на -о
    (r'.*е$', 'ADV'),            # наречия на -е
    (r'.*но$', 'ADV'),           # наречия на -но

    # Числительные
    (r'^-?[0-9]+(\.[0-9]+)?$', 'NUM'),  # целые и десятичные числа

    # Местоимения
    (r'.*ый$', 'PRON'),           # личные местоимения в форме 2-го лица ед.ч.
    (r'.*ая$', 'PRON'),           # личные местоимения в форме 2-го лица ед.ч.
    (r'.*ое$', 'PRON'),           # личные местоимения в форме 2-го лица ед.ч.
    (r'.*ые$', 'PRON'),           # личные местоимения в форме 2-го лица ед.ч.
    (r'.*ого$', 'PRON'),          # личные местоимения в форме 2-го лица ед.ч.
    (r'.*он$', 'PRON'),           # личные местоимения в форме 3-го лица ед.ч.
    (r'.*она$', 'PRON'),          # личные местоимения в форме 3-го лица ед.ч.
    (r'.*они$', 'PRON'),          # личные местоимения в форме мн.ч.
    (r'.*их$', 'PRON'),           # личные местоимения в форме родительного падежа

    # Предлоги
    (r'.*в$', 'ADP'),             # предлоги на -в
    (r'.*на$', 'ADP'),            # предлоги на -на
    (r'.*с$', 'ADP'),             # предлоги на -с
    (r'.*к$', 'ADP'),             # предлоги на -к
    (r'.*по$', 'ADP'),            # предлоги на -по
    (r'.*за$', 'ADP'),            # предлоги на -за
    (r'.*перед$', 'ADP'),         # предлоги на -перед
    (r'.*над$', 'ADP'),           # предлоги на -над
    (r'.*под$', 'ADP'),           # предлоги на -под
    (r'.*между$', 'ADP'),         # предлоги на -между

    # Причастия
    (r'.*ущий$', 'ADJ'),         # причастия на -ущий
    (r'.*ющая$', 'ADJ'),         # причастия на -ющая
    (r'.*ющее$', 'ADJ'),         # причастия на -ющее
    (r'.*ющие$', 'ADJ'),         # причастия на -ющие
    (r'.*емый$', 'ADJ'),         # причастия на -емый
    (r'.*омая$', 'ADJ'),         # причастия на -омая
    (r'.*имое$', 'ADJ'),         # причастия на -имое
    (r'.*имые$', 'ADJ'),         # причастия на -имые
    (r'.*енный$', 'ADJ'),        # причастия на -енный
    (r'.*енная$', 'ADJ'),        # причастия на -енная
    (r'.*енное$', 'ADJ'),        # причастия на -енное
    (r'.*енные$', 'ADJ'),        # причастия на -енные

    (r'[,.:;?!]', 'PUNCT'),                        # Знаки пунктуации
    (r'\b(?:он|когда|что|они)\b', 'PRON'),         # Местоимения
    (r'\b(?:когда)\b', 'SCONJ'),                   # Подчинительные союзы
    (r'\b(?:—)\b', 'PUNCT'),                       # Знаки пунктуации
    (r'\b(?:быть)\b', 'AUX'),                       # Вспомогательные глаголы

    # Остальные слова
    (r'.*', 'NOUN')                # существительные (по умолчанию)
]


In [54]:
regexp_tagger = RegexpTagger(patterns)
print("RegexpTagger accuracy:", regexp_tagger.evaluate(test_sent))

RegexpTagger accuracy: 0.4198761109614867


Коэффициент точности (accuracy) 0.4198 для RegexpTagger означает, что использованные регулярные выражения могут не охватывать все возможные случаи, что приводит к низкой точности.

## Задание 2.
- Проверить, насколько хорошо работает NER
- Данные брать из Index of /pub/named_entities
- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
- передаём в сетку токен и его соседей.
- передаём в сетку только токен.
- свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

In [56]:
!wget https://github.com/dice-group/FOX/raw/master/input/Wikiner/aij-wikiner-ru-wp3.bz2

--2023-07-28 13:33:55--  https://github.com/dice-group/FOX/raw/master/input/Wikiner/aij-wikiner-ru-wp3.bz2
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dice-group/FOX/master/input/Wikiner/aij-wikiner-ru-wp3.bz2 [following]
--2023-07-28 13:33:55--  https://raw.githubusercontent.com/dice-group/FOX/master/input/Wikiner/aij-wikiner-ru-wp3.bz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7856559 (7.5M) [application/octet-stream]
Saving to: ‘aij-wikiner-ru-wp3.bz2’

aij-wikiner-ru-wp3. 100%[===================>]   7.49M  --.-KB/s    in 0.1s    

2023-07-28 13:33:56 (69.0 MB/s) - ‘aij-wikiner-ru-

In [61]:
from corus import load_wikiner

path = 'aij-wikiner-ru-wp3.bz2'
records = load_wikiner(path)
# next(records)

In [63]:
# Создание списка предложений с размеченными сущностями в формате, используемом в nltk
sentences = []
for record in records:
    tokens = [(token.text, token.pos, token.tag) for token in record.tokens]
    sentences.append(tokens)

# Разделение данных на обучающую и тестовую выборки
train_size = int(0.8 * len(sentences))
train_sent = sentences[:train_size]
test_sent = sentences[train_size:]


In [66]:
# !pip install spacy
# !python -m spacy download ru_core_news_sm

2023-07-28 13:41:45.394195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 939.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 39.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=b5e4de070b7035bdb8b2019054a24c0aba23efd8104908f14b65b1b1362f7d56
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [68]:
# Пример использования spaCy для NER на русском языке
import spacy

nlp = spacy.load("ru_core_news_sm")
text = "Шимон Перес – единственный израильтянин, который занимал как должность Президента страны, так и должность премьер-министра Израиля (дважды)."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)


Шимон Перес PER
Израиля LOC


In [112]:
# Для разметки NER с помощью NLTK сначала производим токенизацию слов, затем POS тэггинг.
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = 'Нетаньяху объявил утром 29 марта, что Израиль, как ожидается, сможет присоединиться к программе безвизового въезда в США в сентябре 2023 года.'

nltk.pos_tag(nltk.word_tokenize(document))

[('Нетаньяху', 'NOUN'),
 ('объявил', 'VERB'),
 ('утром', 'NOUN'),
 ('29', 'CD'),
 ('марта', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('Израиль', 'PROPN'),
 (',', 'PUNCT'),
 ('как', 'SCONJ'),
 ('ожидается', 'VERB'),
 (',', 'PUNCT'),
 ('сможет', 'VERB'),
 ('присоединиться', 'VERB'),
 ('к', 'ADP'),
 ('программе', 'NOUN'),
 ('безвизового', 'ADJ'),
 ('въезда', 'NOUN'),
 ('в', 'ADP'),
 ('США', 'PROPN'),
 ('в', 'ADP'),
 ('сентябре', 'NOUN'),
 ('2023', 'ADJ'),
 ('года', 'NOUN'),
 ('.', 'PUNCT')]

Свой NER

In [124]:
import re

def rule_based_ner(text):
    # Define regular expressions for different entity types
    date_pattern = (
        r'\b\d{1,2}/\d{1,2}/\d{2,4}\b' +
        r'|' + r'\b\d{1,2}-\d{1,2}-\d{2,4}\b' +
        r'|' + r'\b\d{1,2}\.\d{1,2}\.\d{2,4}\b' +
        r'|' + r'\b\d{4}-\d{1,2}-\d{1,2}\b' +
        r'|' + r'\b\d{1,2}/\d{1,2}\b' +
        r'|' + r'\b\d{1,2}-\d{1,2}\b' +
        r'|' + r'\b\d{1,2}\.\d{1,2}\b' +
        r'|' + r'\b\d{1,2}\s(?:января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)(?:\s\d{2,4})?\b' +
        r'|' + r'\b\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec)\s\d{2,4}\b'
    )

    # Find all matches for each pattern
    date_entities = re.findall(date_pattern, text)

    # Create a dictionary to store the entities and their types
    entities = {}
    entities['DATE'] = date_entities

    return entities



text = "28 Sept 2016 — Рано утром 28 сентября на 94-м году жизни в медицинском центре 'Шиба' (Тель а-Шомер) скончался бывший президент Израиля ШИмон Перес."
entities = rule_based_ner(text)
print(entities)


{'DATE': ['28 Sept 2016', '28 сентября']}
